In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_theme(style="ticks")

In [ ]:
load = pd.read_parquet("../data/energy.parquet").asfreq("h")

In [ ]:
sns.lineplot(load)

In [ ]:
sns.lineplot(load.query("date_time.dt.year == 2023"))

## Seasonalities

In [ ]:
# groupby month and show grouped boxplot
load["month"] = load.index.month
load["day"] = load.index.day
load["hour"] = load.index.hour
load["weekday"] = load.index.weekday
sns.boxplot(x="month", y="load", data=load)

In [ ]:
sns.boxplot(x="weekday", y="load", data=load)

In [ ]:
sns.boxplot(x="hour", y="load", data=load)

## Removing the yearly seasonality

In [ ]:
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.transformations.series.detrend import Deseasonalizer, Detrender

model = ExponentialSmoothing(sp=52 * 24 * 7)
detrender = Detrender(forecaster=model)

detrender.fit(load["load"])

In [ ]:
detrender.transform(load["load"]).loc["2023"].plot()

In [ ]:
Deseasonalizer(sp=52 * 24 * 7).fit_transform(load["load"]).loc["2019"].plot()

In [ ]:
y = load.loc["2019":, "load"]
y.info()

In [ ]:
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.forecasting.darts import DartsLinearRegressionModel
from sktime.transformations.series.boxcox import LogTransformer

# from sktime.transformations.series.detrend import Deseasonalizer


quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]
model = DartsLinearRegressionModel(
    lags=24,
    output_chunk_length=24,
    likelihood="quantile",
    quantiles=quantiles,
    multi_models=False,
    kwargs=dict(solver="highs-ipm"),
)
# model = DartsXGBModel(
#         lags=24*7,
#         output_chunk_length=24,
#         likelihood="quantile",
#         quantiles=quantiles,
#         multi_models=False,
#         kwargs=dict(n_jobs=1)
# )

forecaster = TransformedTargetForecaster(
    [
        LogTransformer(),
        # Detrender(ExponentialSmoothing(sp=52 * 24 * 7)),
        # Deseasonalizer(sp=52 * 24 * 7),
        model,
    ]
)
forecaster

In [ ]:
from sktime.forecasting.compose import ForecastingPipeline
from sktime.transformations.compose import YtoX
from sktime.transformations.series.date import DateTimeFeatures
from sktime.transformations.series.fourier import FourierFeatures

# from sktime.transformations.series.holiday import CountryHolidaysTransformer

pipe = ForecastingPipeline(
    [
        YtoX(),
        DateTimeFeatures(ts_freq="H", manual_selection=["is_weekend"]),
        FourierFeatures(sp_list=[24, 24 * 7], fourier_terms_list=[1, 1], freq="h"),
        forecaster,
    ]
)
pipe.fit(y.loc["2022":])

In [ ]:
from probafcst.plotting import plot_quantiles

y_pred = pipe.predict_quantiles(np.arange(1, 25), alpha=quantiles)
plot_quantiles(y.iloc[-24 * 7 :], y_pred)

In [ ]:
from probafcst.backtest import backtest, get_window_params

wdw = get_window_params(
    n_years_initial_window=2, step_length_days=90, forecast_steps_days=1, freq="h"
)
result = backtest(
    pipe, y, **wdw, quantiles=quantiles, backend="loky", splitter_type="sliding"
)

In [ ]:
for i, (_, y_test, y_pred_quantiles) in result[2].iloc[::4].iterrows():
    plot_quantiles(y_test, y_pred_quantiles)